## Creating the SQLite Databases Necessary for DP1 Sorcha

This notebook creates the SQLite databases needed to run **DP1** in Sorcha. This code has been created to run in the Rubin Science Platform notebooks to obtain the databases. It creates two files and outputs the queries needed for the Sorcha config file:

1. **Visits database** from `CcdVisit`
2. **Pointing database** from `Visit`

---
For running DP1, obtain the databases using this notebook, run the command line arg `sorcha init` and select the **DP1 config file** option. Then run the code with:

sorcha run -c DP1_visits_footprint.ini --pd dp1_pointing.db --ob orbits_filename.csv -p colours_filename.csv -o ./ -s dp1 --vd dp1_ccdvisits.db 

`orbits_filename.csv` and `colours_filename.csv` should be your input files for object's orbits and colours, respectively.


In [1]:
import sqlite3
from lsst.rsp import get_tap_service
from lsst.daf.butler import Butler
from rubin_sim.phot_utils import fwhm_geom2_fwhm_eff
import numpy as np

def create_visits_database(query, sqlite_db_file= "dp1_ccdvisits.db", table_name='observations',service = get_tap_service("tap"), butler = Butler('dp1', collections="LSSTComCam/DP1")):
    """
    Create a SQLite ccd visits database.
    """

    job = service.submit_job(query)
    job.run()
    job.wait(phases=['COMPLETED', 'ERROR'])
    print('Job phase is', job.phase)
    if job.phase == 'ERROR':
        job.raise_if_error()
    df = job.fetch_result().to_table().to_pandas()
    job.delete()

    conn = sqlite3.connect(sqlite_db_file)
    cursor = conn.cursor()

    cursor.execute(f"DROP TABLE IF EXISTS {table_name}")




    cursor.execute(f'''
    CREATE TABLE {table_name} (
        visitId INTEGER,
        ccdVisitId INTEGER,
        dec REAL,
        ra REAL,
        llcdec REAL,
        llcra REAL,
        lrcdec REAL,
        lrcra REAL,
        ulcdec REAL,
        ulcra REAL,
        urcdec REAL,
        urcra REAL,
        magLim REAL
    )
    ''')

    df.to_sql(table_name, conn, if_exists='append', index=False)
    
    conn.commit()
    
    try:
    
        cursor.execute(
            "CREATE INDEX idx_obs ON observations(visitId, ccdVisitId, llcra, llcdec, lrcra, lrcdec, urcra, urcdec, ulcra, ulcdec, magLim)"
        )
        
        conn.commit()
    except Exception as e:
        print(f"Error creating index: {e}")

    
    conn.close()
    
    print(f"Visits database created at '{sqlite_db_file}' with table '{table_name}'.\n")
    fov_query = f'SELECT llcra, llcdec, lrcra, lrcdec, urcra, urcdec, ulcra, ulcdec, ra as ra_centre, dec as dec_centre, ccdVisitId as detectorID, magLim as fieldFiveSigmaDepth_mag  FROM {table_name} WHERE visitId = ?'
    print("sorcha FOV config variable visits_query will be\n")
    print("    \033[1;32m" + fov_query + "\033[0m\n")


def create_pointing_database(query, seeing_query, sqlite_db_file= "dp1_pointing.db", table_name='observations',service = get_tap_service("tap"), butler = Butler('dp1', collections="LSSTComCam/DP1")):
    """
    Create a SQLite pointing database.

    Parameters
    ----------
    query : str
        query for visits DP1 database.

    sqlite_db_file : str, optional
        Path to the SQLite database file to create or overwrite.

    table_name : str, optional
        Name of the table to create in the SQLite database. Default is 'observations'.

    Returns
    -------
    None
    """

    job = service.submit_job(query)
    job.run()
    job.wait(phases=['COMPLETED', 'ERROR'])
    print('Job phase is', job.phase)
    if job.phase == 'ERROR':
        job.raise_if_error()
    df = job.fetch_result().to_table().to_pandas()
    job.delete()

    conn = sqlite3.connect(sqlite_db_file)
    cursor = conn.cursor()

    cursor.execute(f"DROP TABLE IF EXISTS {table_name}")

    

    job = service.submit_job(seeing_query)
    job.run()
    job.wait(phases=['COMPLETED', 'ERROR'])
    print('Job phase is', job.phase)
    if job.phase == 'ERROR':
        job.raise_if_error()
    df_see = job.fetch_result().to_table().to_pandas()
    job.delete()
    seeing_list = []
    for visitId in df["visit"]:

        seeing_per_id = df_see.loc[df_see["visitId"] == visitId, "seeing"]
        
        seeing = np.mean(seeing_per_id) if not seeing_per_id.empty else np.nan
        seeing_list.append(seeing)
    
    df["seeing"] = seeing_list

    df["effseeing"] = fwhm_geom2_fwhm_eff(df["seeing"])
    cursor.execute(f'''
    CREATE TABLE {table_name} (
        band TEXT,
        expTime INTEGER,
        dec REAL,
        ra REAL,
        obsStart REAL,
        skyRotation REAL,
        visit INTEGER PRIMARY KEY,
        expMidptMJD REAL,
        seeing REAL,
        effseeing REAL
    )
    ''')

    df.to_sql(table_name, conn, if_exists='append', index=False)
    
    conn.commit()
    
    conn.close()
    
    print(f"Pointing database created at '{sqlite_db_file}' with table '{table_name}'.\n")

    
    pointing_query = f'SELECT visit as observationId, expMidptMJD as observationMidpointMJD_TAI, obsStart as visitTime,expTime as visitExposureTime, band as filter,  ra as fieldRA_deg, dec as fieldDec_deg, skyRotation as fieldRotSkyPos_deg, seeing as seeingFwhmGeom_arcsec, effseeing as seeingFwhmEff_arcsec FROM {table_name} order by observationId'
    
    print("sorcha inputs config variable pointing_sql_query will be\n")
    print("    \033[1;32m" + pointing_query + "\033[0m\n")




In [2]:
datarelease = "dp1"

# creating service and butler to query database
service = get_tap_service("tap")
butler = Butler(datarelease, collections="LSSTComCam/DP1")



# visits database query. This will create the ccdvisits.db and give the needed query for Sorcha
visits_query = "SELECT ccdVisitId,dec,llcdec,llcra,lrcdec,lrcra,magLim,ra, ulcdec,ulcra,urcdec,urcra,visitId FROM "+datarelease+".CcdVisit"
create_visits_database(visits_query, service = service, butler = butler)


# Pointing database query. This will create the pointing.db and give the needed query for Sorcha
pointing_query = "SELECT band,dec,expTime,obsStart,ra,skyRotation,visit,expMidptMJD FROM "+datarelease+".Visit"
seeing_query = "SELECT visitId, seeing FROM "+datarelease+".CcdVisit" # used to get average seeing per observation from per ccd. 
create_pointing_database(pointing_query, seeing_query, service = service, butler = butler)


Job phase is COMPLETED
Visits database created at 'dp1_ccdvisits.db' with table 'observations'.

sorcha FOV config variable visits_query will be

    SELECT llcra, llcdec, lrcra, lrcdec, urcra, urcdec, ulcra, ulcdec, ra as ra_centre, dec as dec_centre, ccdVisitId as detectorID, magLim as fieldFiveSigmaDepth_mag  FROM observations WHERE visitId = ?

Job phase is COMPLETED
Job phase is COMPLETED
Pointing database created at 'dp1_pointing.db' with table 'observations'.

sorcha inputs config variable pointing_sql_query will be

    SELECT visit as observationId, expMidptMJD as observationMidpointMJD_TAI, obsStart as visitTime,expTime as visitExposureTime, band as filter,  ra as fieldRA_deg, dec as fieldDec_deg, skyRotation as fieldRotSkyPos_deg, seeing as seeingFwhmGeom_arcsec, effseeing as seeingFwhmEff_arcsec FROM observations order by observationId

